# SquamataAeroMag - Jupyter notebook for creating metadata for Aeromagnetic data-releases to ScienceBase

### Version 1.7

#### Visit https://github.com/pbrown-usgs/SquamataAeroMag/blob/master/SquamataAeroMagEntityAndAttributeArchive.ipynb for Entity and Attribute code from previous Airborne magnetic surveys

This module performs the following operations:
- Create list of data directories.
- Identify files accompanying data release.
- Create file listing for metadata XML markup.
- Identify and load Aeromagnetic *.CSV data file.
- Create file list of data release files
- Create User Editable Keywords Listing
- Create entity and attribute XML markup for all files being released.
- Poplulate metadata template
- Validate metadata; create error log; create HTML and FGDC Text versions of the metadata. (In development - use https://mrdata.usgs.gov/validation/ for validation in the interim)

Known issues needing repair:
- Add items here...

## Future development plans for SquamataSB

- Create all child metadata files from first example created in previous steps. 
- Batch upload files to ScienceBase.
- Batch remove files from ScienceBase. 
- Change ScienceBase parameters such as citation information, add orcid ids, add USGS CMS tags, etc. 

### Instructions
- Create a template xml format that contains boilerplate text common to all childeren in a data release.  Be sure this template contains the approriate curly bracket tags, {SquamataTagExample} used to populate the template using SquamataAeromag.

### To execute a function/command select a cell and Hold-Shift + Press-Enter

**The 'r' signifies a string literal. Use for paths.**

Metadata wizard:  Advanced, Open In a jupyter Notebook?
Metadata Wizard 2.o from ScienceBase

v1.0: Read csv File into pandas
v1.1: Create Array of csv column headers, assign header array of channel descriptions and units, create entity and attribute text for csv file
v1.2 Create file listing, create editable keywords text
v1.3 Creat entity and attributes for files other than CSV
v1.4 Populate metadata template
v1.5 Populate variables with boilerplate text, create file lists, write to metadata template.


In [1]:
# Phil Brown (pbrown@usgs.gov) 2019 Beta
# Working Python 3 Notebook used to facilitate the release of Audio Magnetotelluric (AMT) Data to ScienceBase.

In [2]:
# Test Cell
print ("Jupyter is working.") #To run this cell, hold down Shift and press Enter.

Jupyter is working.


In [184]:
# Load required Libraries
import sys
import os
import zipfile
import csv
#import pysb
import requests
import shutil
from shutil import copyfile
import zipfile
import datetime
import glob
from lxml import etree
import json
import pickle
import shutil
import fileinput
import json
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
from lxml import etree
##from pymdwizard.core.xml_utils import XMLRecord
##from pymdwizard.core.xml_utils import XMLNode
import re
from ipywidgets import *
from IPython.display import display
from IPython.html.widgets import widgets
import datetime
import dateutil.parser
import time
from IPython.display import Javascript

# Collect Information For Metadata Template

In [246]:
# Create an Array of Origin Names between single quotes below
arrOrigin = []

#Change Text Below to the Appropriate Data Release Creators - seperate list items in single quotes ('')
# seperated by a comma (,).
#########################################################################################################################
arrOrigin = ['McCafferty, A.E.', 'Brown, P.J.'] 
#########################################################################################################################
origin = ''
for i in range(len(arrOrigin)):
    origin = origin + '\n\t\t\t\t<origin>' + arrOrigin[i] + '</origin>'

print ('Origin text:') 
print (origin)    

Origin text:

				<origin>McCafferty, A.E.</origin>
				<origin>Brown, P.J.</origin>


###  -- Replace with month, day, year (YYYYMMDD) you request SB page to go live --

In [260]:
#Enter Pub Date between single quotes below

#########################################################################################################################
pubdate = '20200831'
#########################################################################################################################
print (pubdate)

20200831


In [187]:
#Enter Data Release Title between single quotes below

#########################################################################################################################
title = 'Airborne magnetic and radiometric survey over northwest Arkansas, 2019-2020; Magnetic Data'
#########################################################################################################################
print (title)

Airborne magnetic and radiometric survey over northwest Arkansas, 2019-2020; Magnetic Data


In [188]:
#Enter Other Citation between single quotes below

#########################################################################################################################
othercit = 'Additional information about Originators: Anne E. McCafferty, https://orcid.org/0000-0001-5574-9201; Philip J. Brown, http://orcid.org/0000-0002-2415-7462'
#########################################################################################################################
print (othercit)

Additional information about Originators: Anne E. McCafferty, https://orcid.org/0000-0001-5574-9201; Philip J. Brown, http://orcid.org/0000-0002-2415-7462


In [189]:
#Enter Online Link between single quotes below

##########################################################################################################################
onlink = 'https://doi.org/10.5066/P91O2Y8W'
##########################################################################################################################
print (onlink)

https://doi.org/10.5066/P91O2Y8W


### -- Please see minor edits to abstract. Please update the final sentences of the abstract to only refer to the mag data included in this child item --

In [190]:
#Enter Data Release Abstract between single quotes below

###############################################################
abstract = 'This publication provides digital flight line data for a high-resolution magnetic survey over an area of northwest Arkansas. The airborne geophysical survey was jointly funded by the USGS National Cooperative Geologic Mapping Program and the Earth Mapping Resource Initiative and was designed to meet the complimentary needs related to geologic mapping and mineral resource research. The survey covers an irregular area 225 by 110 km over the transition between the Ouachita and Ozarks Plateau physiographic provinces and includes 67,614 line km of magnetic data. Data were collected from a fixed-wing aircraft flown at mean terrain clearance of 167 meters (m) above topography along N-S flight lines spaced at 200-m intervals. Tie lines were flown in an E-W direction every 3000 m. Data were collected by EON Geosciences under contract with the USGS using a fixed wing aircraft with a magnetometer mounted in the tail stinger. The survey operated out of the Branson, Missouri airport from October of 2019 to March of 2020. Files that are available in this child item include flight line data for the magnetic survey.'
print (abstract)

This publication provides digital flight line data for a high-resolution magnetic survey over an area of northwest Arkansas. The airborne geophysical survey was jointly funded by the USGS National Cooperative Geologic Mapping Program and the Earth Mapping Resource Initiative and was designed to meet the complimentary needs related to geologic mapping and mineral resource research. The survey covers an irregular area 225 by 110 km over the transition between the Ouachita and Ozarks Plateau physiographic provinces and includes 67,614 line km of magnetic data. Data were collected from a fixed-wing aircraft flown at mean terrain clearance of 167 meters (m) above topography along N-S flight lines spaced at 200-m intervals. Tie lines were flown in an E-W direction every 3000 m. Data were collected by EON Geosciences under contract with the USGS using a fixed wing aircraft with a magnetometer mounted in the tail stinger. The survey operated out of the Branson, Missouri airport from October of

In [191]:
#Enter Data Release Purpose between single quotes below

#########################################################################################################################
purpose = 'The magnetic survey was flown as a component of U.S. Geological Survey (USGS) projects to improve the geologic framework for research related to mineral resource, energy resource, and geologic hazards studies in the area. Results from the survey will aid in providing insight into the magnetic signatures of exposed and concealed geology.'
#########################################################################################################################
print (purpose)

The magnetic survey was flown as a component of U.S. Geological Survey (USGS) projects to improve the geologic framework for research related to mineral resource, energy resource, and geologic hazards studies in the area. Results from the survey will aid in providing insight into the magnetic signatures of exposed and concealed geology.


In [192]:
#Enter Data Release Supplementary Information between single quotes below

#########################################################################################################################
supplinf = 'References cited: \n'\
+ '\nDay, W.D., 2019, The Earth MappingResources Initiative (Earth MRI): Mapping the nations critical mineral resources: U.S. Geological Survey Fact Sheet 2019-3007, https://doi.org/10.3133/fs20193007. \n'\
+ '\nThis metadata describes the following files:'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_TMFIShaded.png'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMFI.tif'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_TMFI.tif'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_VerticalMagneticGradientShaded.tif'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMF.tif'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_MagneticData.csv'
#########################################################################################################################
print (supplinf)

References cited: 

Day, W.D., 2019, The Earth MappingResources Initiative (Earth MRI): Mapping the nations critical mineral resources: U.S. Geological Survey Fact Sheet 2019-3007, https://doi.org/10.3133/fs20193007. 

This metadata describes the following files:
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_TMFIShaded.png
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMFI.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_TMFI.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_VerticalMagneticGradientShaded.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMF.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_MagneticData.csv


In [193]:
#Enter Data Release Begining Date between single quotes below

#########################################################################################################################
begdate = '20191005'
#########################################################################################################################
print (begdate)

20191005


In [194]:
#Enter Data Release Begining Date between single quotes below

#########################################################################################################################
enddate = '20200307'
#########################################################################################################################
print (enddate)

20200307


In [195]:
#Enter Data Release Access Constraints between single quotes below

#########################################################################################################################
accconst = 'None. Please see '"'Distribution Info'"' for details.'
#########################################################################################################################
print (accconst)

None. Please see 'Distribution Info' for details.


In [196]:
#Enter Data Release Use Constraints between single quotes below

#########################################################################################################################
useconst = 'There is no guarantee concerning the accuracy of the data. Any user who modifies the data is obligated to describe the types of modifications they perform. Data have been checked to ensure the accuracy. If any errors are detected, please notify the originating office. The U.S. Geological Survey strongly recommends that careful attention be paid to the metadata file associated with these data. Acknowledgment of the U.S. Geological Survey would be appreciated in products derived from these data. User specifically agrees not to misrepresent the data, nor to imply that changes made were approved or endorsed by the U.S. Geological Survey. Please refer to http://www.usgs.gov/privacy.html for the USGS disclaimer.'
#########################################################################################################################
print (useconst)

There is no guarantee concerning the accuracy of the data. Any user who modifies the data is obligated to describe the types of modifications they perform. Data have been checked to ensure the accuracy. If any errors are detected, please notify the originating office. The U.S. Geological Survey strongly recommends that careful attention be paid to the metadata file associated with these data. Acknowledgment of the U.S. Geological Survey would be appreciated in products derived from these data. User specifically agrees not to misrepresent the data, nor to imply that changes made were approved or endorsed by the U.S. Geological Survey. Please refer to http://www.usgs.gov/privacy.html for the USGS disclaimer.


In [197]:
#Enter Data Release Data Contact between single quotes below

#########################################################################################################################
cntperDataContact = 'Anne E. McCafferty'
#########################################################################################################################
print (cntperDataContact)

Anne E. McCafferty


In [198]:
#Enter Data Release Data Contact's Position between single quotes below

#########################################################################################################################
cntposDataContact = 'Research Geophysicist'
#########################################################################################################################
print (cntposDataContact)

Research Geophysicist


In [199]:
#Enter Data Release Data Contact's Phone Number between single quotes below

#########################################################################################################################
cntvoiceDataContact = '303-236-1397'
#########################################################################################################################
print (cntvoiceDataContact)

303-236-1397


In [200]:
#Enter Data Release Data Contact's email between single quotes below

#########################################################################################################################
cntemailDataContact = 'anne@usgs.gov'
#########################################################################################################################
print (cntemailDataContact)

anne@usgs.gov


In [201]:
#Enter Data Release Data Contact's Phone Number between single quotes below

#########################################################################################################################
datacred = 'This survey and release of the dataset was funded by the Earth Mapping Resources Initiative (Earth MRI).'
#########################################################################################################################
print (datacred)

This survey and release of the dataset was funded by the Earth Mapping Resources Initiative (Earth MRI).


In [202]:
#Enter Data Release attraccr between single quotes below

#########################################################################################################################
attraccr = 'Daily quality control of raw radiometric and magnetic data was performed by the contractor. Further quality control measures occurred post flight by the contractor and USGS scientist-in-charge before approval and acceptance of the final data. The data have been peer reviewed and compared with related ancillary data.'
#########################################################################################################################
print (attraccr)

Daily quality control of raw radiometric and magnetic data was performed by the contractor. Further quality control measures occurred post flight by the contractor and USGS scientist-in-charge before approval and acceptance of the final data. The data have been peer reviewed and compared with related ancillary data.


In [203]:
#Enter Data Release logic between single quotes below

#########################################################################################################################
logic = 'Data were visually inspected for reasonableness and spatial consistency.'
#########################################################################################################################
print (logic)

Data were visually inspected for reasonableness and spatial consistency.


In [204]:
#Enter Data Release complete between single quotes below

#########################################################################################################################
complete = 'Data set is considered complete for the information presented, as described in the abstract. Users are advised to read the rest of the metadata record carefully for additional details.'
#########################################################################################################################
print (complete)

Data set is considered complete for the information presented, as described in the abstract. Users are advised to read the rest of the metadata record carefully for additional details.


In [205]:
#Enter Data Release horizpar between single quotes below

#########################################################################################################################
horizpar = 'Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.'
#########################################################################################################################
print (horizpar)

Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.


In [206]:
#Enter Data Release vertaccr between single quotes below

#########################################################################################################################
vertaccr = 'Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.'
#########################################################################################################################
print (vertaccr)

Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.


In [207]:
#Enter Data Release latres between single quotes below

#########################################################################################################################
latres = '0.000001'
#########################################################################################################################
print (latres)

0.000001


In [208]:
#Enter Data Release Data longres between single quotes below

#########################################################################################################################
longres = '0.000001'
#########################################################################################################################
print (longres)

0.000001


In [209]:
#Enter Data Release horizdn between single quotes below

#########################################################################################################################
horizdn = 'WGS_1984'
#########################################################################################################################
print (horizdn)


WGS_1984


In [210]:
#Enter Data Release ellips between single quotes below

#########################################################################################################################
ellips = 'WGS_1984'
#########################################################################################################################
print (ellips)

WGS_1984


In [211]:
#Enter Data Release semiaxis between single quotes below

#########################################################################################################################
semiaxis = '6378206.4'
#########################################################################################################################
print (semiaxis)


6378206.4


In [212]:
#Enter Data Release denflat between single quotes below

#########################################################################################################################
denflat = '294.9786982'
#########################################################################################################################
print (denflat)

294.9786982


In [213]:
#Enter Data Release Data Contact's Phone Number between single quotes below

#########################################################################################################################
distliab = 'Unless otherwise stated, all data, metadata and related materials are considered to satisfy the quality standards relative to the purpose for which the data were collected. Although these data and associated metadata have been reviewed for accuracy and completeness and approved for release by the U.S. Geological Survey (USGS), no warranty expressed or implied is made regarding the display or utility of the data on any other system or for general or scientific purposes, nor shall the act of distribution constitute any such warranty.'
#########################################################################################################################
print (distliab)

Unless otherwise stated, all data, metadata and related materials are considered to satisfy the quality standards relative to the purpose for which the data were collected. Although these data and associated metadata have been reviewed for accuracy and completeness and approved for release by the U.S. Geological Survey (USGS), no warranty expressed or implied is made regarding the display or utility of the data on any other system or for general or scientific purposes, nor shall the act of distribution constitute any such warranty.


In [214]:
#Enter Data Release stdorder between single quotes below

#########################################################################################################################
stdorder = 'Not sure if we should continue to add a standard order.  I added this becuase Stu wanted to have something in the metadata that warns of the size of the csv file.  Technically we could write a standard order for all items but is seems like a lot of extra work.'
#########################################################################################################################
print (stdorder)

Not sure if we should continue to add a standard order.  I added this becuase Stu wanted to have something in the metadata that warns of the size of the csv file.  Technically we could write a standard order for all items but is seems like a lot of extra work.


In [215]:
#Enter Data Release metadata creation date between single quotes below

#########################################################################################################################
metd = '20200708'
#########################################################################################################################
print (metd)

20200708


In [216]:
#Enter Data Release cntperMetaInfo between single quotes below

#########################################################################################################################
cntperMetaInfo = 'Philip Brown'
#########################################################################################################################
print (cntperMetaInfo)

Philip Brown


In [217]:
#Enter Data Release cntposMetaInfo between single quotes below

#########################################################################################################################
cntposMetaInfo = 'Geophysicist'
#########################################################################################################################
print (cntposMetaInfo)

Geophysicist


In [218]:
#Enter Data Release cntvoiceMetaInfo between single quotes below

#########################################################################################################################
cntvoiceMetaInfo = '303-236-1310'
#########################################################################################################################
print (cntvoiceMetaInfo)

303-236-1310


In [219]:
#Enter Data Release cntemailMetaInfo between single quotes below

#########################################################################################################################
cntemailMetaInfo = 'pbrown@usgs.gov'
#########################################################################################################################
print (cntemailMetaInfo)

pbrown@usgs.gov


In [220]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrThemeISO = ['geoscientificInformation','economy']
#########################################################################################################################

txtThemeISO = '\t\t\t<theme>\n\t\t\t\t<themekt>ISO 19115 Topic Category</themekt>'
for i in range(len(arrThemeISO)):
 txtThemeISO = txtThemeISO +  '\n\t\t\t\t<themekey>' + arrThemeISO[i] + '</themekey>'

txtThemeISO = txtThemeISO + '\n\t\t\t</theme>'
print(str(txtThemeISO))


			<theme>
				<themekt>ISO 19115 Topic Category</themekt>
				<themekey>geoscientificInformation</themekey>
				<themekey>economy</themekey>
			</theme>


In [221]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrThemeNONE = ['Mineral Resources Program','MRP', 'Geology, Geophysics, and Geochemistry Science Center' \
            'GGGSC','Earth Mapping Resources Initiative','EarthMRI','horizontal magnetic gradiometry', \
            'vertical magnetic gradient','airborne geophysical survey','U.S. Geological Survey','USGS', \
            'Association of American State Geologists','AASG','EON Geosciences, Inc.','radon','Rn', \
            'Ouachita Plateau','Ozarks Plateau','USA','United States of America' ]
#########################################################################################################################

txtThemeNONE = '\n\t\t\t<theme>\n\t\t\t\t<themekt>None</themekt>'
for i in range(len(arrThemeNONE)):
 txtThemeNONE = txtThemeNONE +  '\n\t\t\t\t<themekey>' + arrThemeNONE[i] + '</themekey>'

txtThemeNONE = txtThemeNONE + '\n\t\t\t</theme>'
print(str(txtThemeNONE))


			<theme>
				<themekt>None</themekt>
				<themekey>Mineral Resources Program</themekey>
				<themekey>MRP</themekey>
				<themekey>Geology, Geophysics, and Geochemistry Science CenterGGGSC</themekey>
				<themekey>Earth Mapping Resources Initiative</themekey>
				<themekey>EarthMRI</themekey>
				<themekey>horizontal magnetic gradiometry</themekey>
				<themekey>vertical magnetic gradient</themekey>
				<themekey>airborne geophysical survey</themekey>
				<themekey>U.S. Geological Survey</themekey>
				<themekey>USGS</themekey>
				<themekey>Association of American State Geologists</themekey>
				<themekey>AASG</themekey>
				<themekey>EON Geosciences, Inc.</themekey>
				<themekey>radon</themekey>
				<themekey>Rn</themekey>
				<themekey>Ouachita Plateau</themekey>
				<themekey>Ozarks Plateau</themekey>
				<themekey>USA</themekey>
				<themekey>United States of America</themekey>
			</theme>


In [222]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrThemeUSGS = ['magnetic field (earth)','geophysics', 'geospatial datasets', \
            'aeromagnetic surveying','aeroradiometric surveying','potassium','uranium', \
            'thorium']
#########################################################################################################################

txtThemeUSGS = '\n\t\t\t<theme>\n\t\t\t\t<themekt>USGS Thesaurus</themekt>'
for i in range(len(arrThemeUSGS)):
 txtThemeUSGS = txtThemeUSGS +  '\n\t\t\t\t<themekey>' + arrThemeUSGS[i] + '</themekey>'

txtThemeUSGS = txtThemeUSGS + '\n\t\t\t</theme>'
print(str(txtThemeUSGS))


			<theme>
				<themekt>USGS Thesaurus</themekt>
				<themekey>magnetic field (earth)</themekey>
				<themekey>geophysics</themekey>
				<themekey>geospatial datasets</themekey>
				<themekey>aeromagnetic surveying</themekey>
				<themekey>aeroradiometric surveying</themekey>
				<themekey>potassium</themekey>
				<themekey>uranium</themekey>
				<themekey>thorium</themekey>
			</theme>


### -- Please add Johnson County, Van Buren County, and Barry County --

In [223]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrPlaceGNIS = ['Boston Mountains','Carroll County', 'Madison County', \
            'Franklin County','Johnson County','Pope County','Newton County', \
            'Boone County','Marion County','Baxter County', 'Stone County', \
            'Izard County','Sharp County','Independence County','Lawrence County', \
            'Searcy County','Washington County', 'Johnson County','Van Buren County','Barry County']
#########################################################################################################################

txtPlaceGNIS = '\n\t\t\t<place>\n\t\t\t\t<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov </placekt>'
for i in range(len(arrPlaceGNIS)):
 txtPlaceGNIS = txtPlaceGNIS +  '\n\t\t\t\t<placekey>' + arrPlaceGNIS[i] + '</placekey>'


txtPlaceGNIS = txtPlaceGNIS + '\n\t\t\t</place>'
print(str(txtPlaceGNIS))


			<place>
				<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov </placekt>
				<placekey>Boston Mountains</placekey>
				<placekey>Carroll County</placekey>
				<placekey>Madison County</placekey>
				<placekey>Franklin County</placekey>
				<placekey>Johnson County</placekey>
				<placekey>Pope County</placekey>
				<placekey>Newton County</placekey>
				<placekey>Boone County</placekey>
				<placekey>Marion County</placekey>
				<placekey>Baxter County</placekey>
				<placekey>Stone County</placekey>
				<placekey>Izard County</placekey>
				<placekey>Sharp County</placekey>
				<placekey>Independence County</placekey>
				<placekey>Lawrence County</placekey>
				<placekey>Searcy County</placekey>
				<placekey>Washington County</placekey>
				<placekey>Johnson County</placekey>
				<placekey>Van Buren County</placekey>
				<placekey>Barry County</placekey>
			</place>


In [224]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrPlaceGNIS = ['Arkansas', 'United States']
#########################################################################################################################

txtPlaceGNIS = '\n\t\t\t<place>\n\t\t\t\t<placekt>Common geographic areas</placekt>'
for i in range(len(arrPlaceGNIS)):
 txtPlaceGNIS = txtPlaceGNIS +  '\n\t\t\t\t<placekey>' + arrPlaceGNIS[i] + '</placekey>'

txtPlaceGNIS = txtPlaceGNIS + '\n\t\t\t</place>'
print(str(txtPlaceGNIS))


			<place>
				<placekt>Common geographic areas</placekt>
				<placekey>Arkansas</placekey>
				<placekey>United States</placekey>
			</place>


In [225]:
## Create editable keywords example.  
## Example text is created after running this cell
## This text is displayed by running "display(keywords) below
keywords = '\n\t\t<keywords>\n' + txtThemeISO + txtThemeNONE + txtThemeUSGS + txtPlaceGNIS \
 + txtPlaceGNIS + '\n\t\t</keywords>'
print (keywords)


		<keywords>
			<theme>
				<themekt>ISO 19115 Topic Category</themekt>
				<themekey>geoscientificInformation</themekey>
				<themekey>economy</themekey>
			</theme>
			<theme>
				<themekt>None</themekt>
				<themekey>Mineral Resources Program</themekey>
				<themekey>MRP</themekey>
				<themekey>Geology, Geophysics, and Geochemistry Science CenterGGGSC</themekey>
				<themekey>Earth Mapping Resources Initiative</themekey>
				<themekey>EarthMRI</themekey>
				<themekey>horizontal magnetic gradiometry</themekey>
				<themekey>vertical magnetic gradient</themekey>
				<themekey>airborne geophysical survey</themekey>
				<themekey>U.S. Geological Survey</themekey>
				<themekey>USGS</themekey>
				<themekey>Association of American State Geologists</themekey>
				<themekey>AASG</themekey>
				<themekey>EON Geosciences, Inc.</themekey>
				<themekey>radon</themekey>
				<themekey>Rn</themekey>
				<themekey>Ouachita Plateau</themekey>
				<themekey>Ozarks Plateau</themekey>
				<themeke

# Create process descriptions and dates

In [300]:
#Now let's create the lineage 2-D array of Process steps and dates
#Add text below in single quotes seperated by commas.  Sequencial 2-D arrays are seperated by brackets []
#2-D Array form is [['procdesc1','procdate1'],['procdesc2','procdate2'],etc...]

######################################################################################################################################################################################################################################################################################################################################################################################
arrLineage = [ \
             ['The airborne survey was flown under contract to the U.S. Geological Survey by EON Geosciences Inc., from October 5, 2019 to March 7, 2020. The magnetic data were acquired with a Geometrics G-822A cesium-vapour split-beam sensor. The radiometric data were acquired with a Radiation Solutions RS-500 series multi-channel gamma-ray spectrometers, models RSX-5 and RSX-4, with 1,024 channels, for a total volume of 33.6 liters downward-looking and 8.4 liters (C-FEON) or 4.2 liters (C-GPTB) upward-looking crystals (for measuring effects of atmospheric radon). Data were collected along east-west flight lines spaced 200 meters (m) apart with north-south tie lines flown every 3,000 meters. A mean terrain clearance of 80 m was maintained except where safety dictated a higher elevation. A total of 66,477 line-kilometers of data were collected.', '20191005'] \
             ,['Preliminary magnetic and radiometric data processing was undertaken by EON Geosciences, Inc. of Montreal, Quebec, Canada. This processing workflow is described in the contractors report included with this data release.', '20200316'] \
             ,['Final data and contractors report were accepted and received by USGS from EON Geosciences, Inc.','20200629'] \
             ,['Created georeferenced rasters of the data grids provided by the contractor.  Created Esri shapefiles of area footprint and line paths for the survey using the line data provided by the contractor.','20200707'] \
             ,['Final data were exported to CSV-format to meet open data requirements.','20200707']
             ]
######################################################################################################################################################################################################################################################################################################################################################################################

txtLineage = '\t\t<lineage>'
for i in range(len(arrLineage)):
 txtLineage = txtLineage + '\n\t\t\t<procstep>' +  '\n\t\t\t\t<procdesc>' + arrLineage[i][0] + '</procdesc>\n\t\t\t\t' \
    + '<procdate>' + arrLineage[i][1] + '</procdate>' + '\n\t\t\t</procstep>'

txtLineage = txtLineage + '\n\t\t</lineage>'
print(str(txtLineage))
lineage = txtLineage

		<lineage>
			<procstep>
				<procdesc>The airborne survey was flown under contract to the U.S. Geological Survey by EON Geosciences Inc., from October 5, 2019 to March 7, 2020. The magnetic data were acquired with a Geometrics G-822A cesium-vapour split-beam sensor. The radiometric data were acquired with a Radiation Solutions RS-500 series multi-channel gamma-ray spectrometers, models RSX-5 and RSX-4, with 1,024 channels, for a total volume of 33.6 liters downward-looking and 8.4 liters (C-FEON) or 4.2 liters (C-GPTB) upward-looking crystals (for measuring effects of atmospheric radon). Data were collected along east-west flight lines spaced 200 meters (m) apart with north-south tie lines flown every 3,000 meters. A mean terrain clearance of 80 m was maintained except where safety dictated a higher elevation. A total of 66,477 line-kilometers of data were collected.</procdesc>
				<procdate>20191005</procdate>
			</procstep>
			<procstep>
				<procdesc>Preliminary magnetic and radio

# Step One - Set Directory Paths
## Please set directory paths below
### Directory paths include
- Data Path: This is the path to the data, data structure should have a directory for each station
- Template Path: The path to the XML metadata template being used for the data.  This template should already include all information common to all child metadata files e.g. originators, larger work citation, etc.

In [227]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
# The r signifies a string literal. Use for paths.

magMetaDataTemplatePath = r"C:\CurrentWork\DataReleases\NorthwesternArkansas\Metadata"
magMetaDataTemplateName = "USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Magnetic--TEMPLATE.xml"
magCSVfileDir = r"C:\CurrentWork\DataReleases\NorthwesternArkansas\CSV_Data"
magCSVfileName = "USA_Northwest-Arkansas-Airborne-Survey-2019-2020_MagneticData.csv"

In [228]:
#Check Paths for the fun of it
magCSVfilePath = magCSVfileDir + '\\' + magCSVfileName
print ('The Mag CSV Data Path is: ' + '"' + magCSVfilePath + '"')#magCSVfilePath
magMetadataTemplatefilePath = magMetaDataTemplatePath + "\\" + magMetaDataTemplateName
print ('The Template File Path is: ' + '"' + magMetadataTemplatefilePath + '"')

The Mag CSV Data Path is: "C:\CurrentWork\DataReleases\NorthwesternArkansas\CSV_Data\USA_Northwest-Arkansas-Airborne-Survey-2019-2020_MagneticData.csv"
The Template File Path is: "C:\CurrentWork\DataReleases\NorthwesternArkansas\Metadata\USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Magnetic--TEMPLATE.xml"


In [229]:
tifFilesPath = r"C:\CurrentWork\DataReleases\NorthwesternArkansas\Geotiffs"
tifList = glob.glob(os.path.join(tifFilesPath, '*.tif'),  recursive=True)
fileListing = ''
rspFileListing = []
for i in range(len(tifList)):
    splitTifList = tifList[i].split('\\')
    fileListing = fileListing +  splitTifList[len(splitTifList) - 1] + '\n'
    rspFileListing.append(splitTifList[len(splitTifList) - 1])
print (fileListing)

USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMF.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMFI.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_TMFI.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_VerticalMagneticGradientShaded.tif



## Lets now import and index values from the CSV file
- We need create arrays of description and unit values for the metadata template.  
- We also want to run stats on the csv columns for the entity and attributes section

In [230]:
# read file into pandas; note that can take over 1 minute depending on the size of the CSV file and computing power.
print ('!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!')
print ('Reading file into Pandas, this may take several minutes depending on the file size...')

magDataFrame = pd.read_csv(magCSVfilePath, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
magDataFrame

!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!
Reading file into Pandas, this may take several minutes depending on the file size...


,baltl,baltlc,barom,baseB,baseBo,date,directn,drape,DTMc,DTMo,...,raltl,raltlc,resmag,topoSRTM,totmag,um3l,x,y,year,z
0,434.727020,434.463745,434.463745,50849.468750,50849.460938,2019/11/01,180,561.883484,453.275665,450.339722,...,115.773071,112.837120,47.682865,457.549774,50667.148438,50681.457031,426297.15625,4012245.75,2019,568.612793
1,434.630951,434.388794,434.388794,50849.468750,50849.460938,2019/11/01,180,561.773315,452.800385,449.706970,...,115.987556,113.068268,47.768021,456.893555,50667.195312,50681.480469,426297.03125,4012238.25,2019,568.531555
2,434.534912,434.313812,434.313812,50849.468750,50849.464844,2019/11/01,180,561.663208,452.325073,449.074219,...,115.573914,113.343529,47.857082,456.237305,50667.246094,50681.507812,426296.90625,4012230.75,2019,568.450317
3,434.438843,434.238861,434.238861,50849.468750,50849.468750,2019/11/01,180,561.553040,451.849762,448.441498,...,116.830132,113.683571,47.950050,455.581055,50667.300781,50681.539062,426296.75000,4012223.25,2019,568.369080
4,434.342804,434.163879,434.163879,50849.468750,50849.468750,2019/11/01,180,561.442932,451.374481,447.808746,...,116.447144,114.088440,48.046921,454.924835,50667.359375,50681.574219,426296.62500,4012215.75,2019,568.287842
5,434.246735,434.088928,434.088928,50849.468750,50849.468750,2019/11/01,180,561.332764,450.899170,447.175995,...,117.059937,114.536728,48.143795,454.268616,50667.417969,50681.617188,426296.50000,4012208.00,2019,568.206665
6,434.150665,434.013977,434.013977,50849.468750,50849.472656,2019/11/01,180,561.222595,450.423889,446.543243,...,116.722893,114.999191,48.244576,453.612366,50667.480469,50681.667969,426296.37500,4012200.50,2019,568.125427
7,434.054626,433.938995,433.938995,50849.468750,50849.476562,2019/11/01,180,561.112488,449.948608,445.910492,...,117.841248,115.455612,48.345356,452.956146,50667.542969,50681.722656,426296.25000,4012193.00,2019,568.044189
8,433.958557,433.864044,433.864044,50849.468750,50849.476562,2019/11/01,180,561.002319,449.473297,445.277771,...,118.699158,115.901619,48.446136,452.299896,50667.605469,50681.785156,426296.09375,4012185.50,2019,567.962952
9,433.862518,433.789062,433.789062,50849.468750,50849.476562,2019/11/01,180,560.892212,448.997986,444.645020,...,120.277107,116.342010,48.543007,451.643646,50667.664062,50681.851562,426295.96875,4012178.00,2019,567.881714


### EandA Channel Templates for the magnetic data are available at the following link to a shared Jupyter Notebook
https://github.com/pbrown-usgs/SquamataAeroMag/blob/master/SquamataAeroMagEntityAndAttributeArchive.ipynb


In [301]:
# now get an array of the header names
magHeaderList = list(magDataFrame.columns.values)
magHeaderString = ''
for i in range(len(magHeaderList)):
 magHeaderString = magHeaderString + magHeaderList[i] + ','

print (magHeaderString)

baltl,baltlc,barom,baseB,baseBo,date,directn,drape,DTMc,DTMo,fid10,fiducial,flt,hgps,jul_day,lat,latitude,line,line_no,lon,longitud,m3l,mfluxX,mfluxY,mfluxZ,mreslc,mreslcb,mreslct,mreslvl,mreslvld,mreslvldi,radar,raltl,raltlc,resmag,topoSRTM,totmag,um3l,x,y,year,z,


In [302]:
# Get Min and Max Latitude and Longitude
#---------------------------------------------------------------------------------
#*************!!!MAKE SURE LAT AND LONG CHANNELS ARE CORRECT BELOW!!!*************
#********* ENTER CORRECT LAT/LONG CHANNEL NAMES BETWEEN SINGLE QUOTES ************
#*****!!!Make sure the results are rounded to the correct # of Sig Figs!!!********
#---------------------------------------------------------------------------------

########################################################################################################################
westbc = magDataFrame['longitud'].dropna().max()
########################################################################################################################
westbc = float(westbc)    # convert to float so that number can be rounded 
westbc = round(westbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

#########################################################################################################################
eastbc = magDataFrame['longitud'].dropna().min()
#########################################################################################################################
eastbc = float(eastbc)    # convert to float so that number can be rounded  
eastbc = round(eastbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

#########################################################################################################################
northbc = magDataFrame['latitude'].dropna().max()
#########################################################################################################################
northbc = float(northbc)    # convert to float so that number can be rounded 
northbc = round(northbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

#########################################################################################################################
southbc = magDataFrame['latitude'].dropna().min()
#########################################################################################################################
southbc = float(southbc)    # convert to float so that number can be rounded  
southbc = round(southbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

print ('westbc: ' + str(westbc))
print ('eastbc: ' + str(eastbc))
print ('northbc: ' + str(northbc))
print ('southbc: ' + str(southbc))

westbc: -91.318886
eastbc: -93.825821
northbc: 36.500267
southbc: 35.513279


In [303]:
#Create Entity and Attribute Text - substitute vales below with definitions from contractors report:
# EandA Channel Templates for the magnetic data are available at the following link to a shared Jupyter Notebook
# https://github.com/pbrown-usgs/SquamataAeroMag/blob/master/SquamataAeroMagEntityAndAttributeArchive.ipynb

#Create Entity and Attribute Text FOR MAG:
csvAttributes = ''
csvEntityDescription = []
csvEntityUnit = []
csvMax = []
csvMin = []
#=======================================================================================================================
# Add entity type and detailed tags
# Below is the entity type description for the Down-Raw spectrum waveforms
entAttText = '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>' + magCSVfileName + '</enttypl>\n' \
  + '\t\t\t\t<enttypd>Table containing attribute information associated with the data set.</enttypd>' \
  + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>'

##-----------------Start EandA here
csvEntityDescription.append("Baro altitude, raw. MSL, computed from measured pressure and temperature")#baltl
csvEntityUnit.append("meters")#baltl
#=======================================================================================================================
csvEntityDescription.append("Baro altitude, edited. MSL, low-pass filtered against noise and spikes")#baltlc
csvEntityUnit.append("meters")#baltlc
#=======================================================================================================================
csvEntityDescription.append("Barometric altitude above MSL. Same as [baltlc]")#barom
csvEntityUnit.append("meters")#barom
#=======================================================================================================================
csvEntityDescription.append("Main mag base, edited. Edited against cultural signal and noise")#baseB
csvEntityUnit.append("nanoteslas")#baseB
#=======================================================================================================================
csvEntityDescription.append("Main mag base, raw	Raw main mag base station")#baseBo
csvEntityUnit.append("nanoteslas")#baseBo
#=======================================================================================================================
csvEntityDescription.append("Acquisition date. Start of acquisition, UTC")#date
csvEntityUnit.append("yyyy/mm/dd")#date
#=======================================================================================================================
csvEntityDescription.append("Flight line direction, azimuth degrees from north.")#directn
csvEntityUnit.append("degrees")#directn
#=======================================================================================================================
csvEntityDescription.append("Drape altitude. MSL, planned flight surface for navigation")#drape
csvEntityUnit.append("meters")#drape
#=======================================================================================================================
csvEntityDescription.append("Digital terrain model. MSL, DTM model, [z-2.5m-raltlc], GPS ant. 2.5m above radar ant.")#DTMc
csvEntityUnit.append("meters")#DTMc
#=======================================================================================================================
csvEntityDescription.append("Digital terrain model. MSL, DTM model, [z-2.5m-raltl], GPS ant. 2.5m above radar ant.")#DTMo
csvEntityUnit.append("meters")#DTMo
#=======================================================================================================================
csvEntityDescription.append("Fiducial, Oasis. RMS system fiducial")#fid10
csvEntityUnit.append("sec")#fid10
#=======================================================================================================================
csvEntityDescription.append("Fiducial number, same as [fid10] (10Hz version)")#fiducial
csvEntityUnit.append("sec")#fiducial
#=======================================================================================================================
csvEntityDescription.append("Flight number")#flt
csvEntityUnit.append("none")#flt
#=======================================================================================================================
csvEntityDescription.append("hgps")#hgps
csvEntityUnit.append("UTC hour")#hgps
#=======================================================================================================================
csvEntityDescription.append("Julian day flown.")#jul_day
csvEntityUnit.append("ddd")#jul_day
#=======================================================================================================================
csvEntityDescription.append("Latitude, decimal degrees.")#lat
csvEntityUnit.append("degrees")#lat
#=======================================================================================================================
csvEntityDescription.append("Latitude, decimal degrees, same as [lat].")#latitude
csvEntityUnit.append("degrees")#latitude
#=======================================================================================================================
csvEntityDescription.append("Line number, edited lll: planned line number, n: segment code.")#line
csvEntityUnit.append("llln")#line
#=======================================================================================================================
csvEntityDescription.append("Flight line number, same as [line].")#line_no
csvEntityUnit.append("llln")#line_no
#=======================================================================================================================
csvEntityDescription.append("Longitude decimal degrees.")#lon
csvEntityUnit.append("degrees")#lon
#=======================================================================================================================
csvEntityDescription.append("Decimal degrees, same as [lon].")#longitud
csvEntityUnit.append("degrees")#longitud
#=======================================================================================================================
csvEntityDescription.append("Compensated TMF, raw. Sensor 3, tail stinger.")#m3l
csvEntityUnit.append("nanoteslas")#m3l
#=======================================================================================================================
csvEntityDescription.append("Fluxgate MF-X, raw. Longitudinal vector fluxgate sensor.")#mfluxX
csvEntityUnit.append("nanoteslas")#mfluxX
#=======================================================================================================================
csvEntityDescription.append("Fluxgate MF-Y, raw. Transverse vector fluxgate sensor.")#mfluxY
csvEntityUnit.append("nanoteslas")#mfluxY
#=======================================================================================================================
csvEntityDescription.append("Fluxgate MF-Z, raw. Vertical vector fluxgate ensor.")#mfluxZ
csvEntityUnit.append("nanoteslas")#mfluxZ
#=======================================================================================================================
csvEntityDescription.append("TMF, partial IGRF removed. Partial IGRF to drape surface removed from m3l.")#mreslc
csvEntityUnit.append("nanoteslas")#mreslc
#=======================================================================================================================
csvEntityDescription.append("TMF, diurnals corrected. Filtered (LP 7500m) and zero-average diurnals removed from mreslc")#mreslcb
csvEntityUnit.append("nanoteslas")#mreslcb
#=======================================================================================================================
csvEntityDescription.append("TMF, Taylor signal removed. Taylor-corrected for drape deviations")#mreslct
csvEntityUnit.append("nanoteslas")#mreslct
#=======================================================================================================================
csvEntityDescription.append("TMF, levelled. Intersection levelling of mreslcb")#mreslvl
csvEntityUnit.append("nanoteslas")#mreslvl
#=======================================================================================================================
csvEntityDescription.append("TMF, micro-levelled Micro-levelling correction applied on mreslct.")#mreslvld
csvEntityUnit.append("nanoteslas")#mreslvld
#=======================================================================================================================
csvEntityDescription.append("IGRF residual of TMF. Using 2015 model with fixed date, 2020-01-01 and altitude 549.89m.")#mreslvldi
csvEntityUnit.append("nanoteslas")#mreslvldi
#=======================================================================================================================
csvEntityDescription.append("Radar altitude above ground, Same as [raltlc], 10Hz version.")#radar
csvEntityUnit.append("meters")#radar
#=======================================================================================================================
csvEntityDescription.append("Radar altitude, raw. AGL, linear transformation from raw readings.")#raltl
csvEntityUnit.append("meters")#raltl
#=======================================================================================================================
csvEntityDescription.append("Radar altitude, edited. AGL, low-pass filtered against noise, spikes and jumps.")#raltlc
csvEntityUnit.append("meters")#raltlc
#=======================================================================================================================
csvEntityDescription.append("CResidual magnetic value. Same as [mreslvldi].")#resmag
csvEntityUnit.append("nanoteslas")#resmag
#=======================================================================================================================
csvEntityDescription.append("Published terrain model MSL, DTM model from SRTM grid.")#topoSRTM
csvEntityUnit.append("nanoteslas")#topoSRTM
#=======================================================================================================================
csvEntityDescription.append("Corrected TMF value. Same as [mreslvld].")#totmag
csvEntityUnit.append("nanoteslas")#totmag
#=======================================================================================================================
csvEntityDescription.append("Uncompensated TMF, raw. Sensor 3, tail stinger.")#um3l
csvEntityUnit.append("nanoteslas")#m3l
#=======================================================================================================================
csvEntityDescription.append("UTM Easting WGS-84, Z15N, differential PP-DGPS.")#x
csvEntityUnit.append("meters")#x
#=======================================================================================================================
csvEntityDescription.append("UTM Northing WGS-84, Z15N, differential PP-DGPS.")#y
csvEntityUnit.append("meters")#y
#=======================================================================================================================
csvEntityDescription.append("year")#year
csvEntityUnit.append("yyyy")#year
#=======================================================================================================================
csvEntityDescription.append("GPS altitude, unedited MSL, differential PP-DGPS.")#z
csvEntityUnit.append("meters")#z
#=======================================================================================================================
##-----------------End EandA here

#Now get a min and max value for all columns

csvHeaderListLength = len(csvEntityDescription)
#print ('Length of Entity Description Array: ' + str(len(csvEntityDescription)))
#print ('Length of Header List: ' + str(len(magHeaderList)))

for i in range(csvHeaderListLength): #csvHeaderListLength
 print (magHeaderList[i] + ': ' + str(i))
 strCol = magHeaderList[i]
 csvMax.append(magDataFrame[strCol].max())
 csvMin.append(magDataFrame[strCol].min())
 print ('Max. Value ' + magHeaderList[i] + ' :' + str(csvMax[i]))
 print ('Min. Value ' + magHeaderList[i] + ' :' + str(csvMin[i]))
 print ('Entt Desc. ' + csvEntityDescription[i])
 print ('Att Unit ' + csvEntityUnit[i] + '\n')
 entAtti = '\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>' + strCol + '</attrlabl>\n' \
        + '\t\t\t\t\t<attrdef>' + csvEntityDescription[i] + '</attrdef>' \
        + '\n\t\t\t\t\t<attrdefs>EON GEOSCIENCES INC.</attrdefs>' \
        + '\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<rdom>\n\t\t\t\t\t\t\t<rdommin>'+ str(csvMin[i]) + '</rdommin>\n\t\t\t\t\t\t\t<rdommax>' \
        + str(csvMax[i]) + '</rdommax>\n\t\t\t\t\t\t\t' + '<attrunit>' + csvEntityUnit[i] + '</attrunit>\n\t\t\t\t\t\t</rdom>\n\t\t\t\t\t\t' \
        + '</attrdomv>\n' \
        + '\t\t\t\t</attr>'
 entAttText = entAttText + entAtti
    
entAttText = entAttText  + '\n\t\t</detailed>\n' 

csvAttributes = entAttText

print ('\nEntity and Attribute Text:\n' + csvAttributes)

baltl: 0
Max. Value baltl :899.5800170898441
Min. Value baltl :14.9452438354492
Entt Desc. Baro altitude, raw. MSL, computed from measured pressure and temperature
Att Unit meters

baltlc: 1
Max. Value baltlc :899.485107421875
Min. Value baltlc :17.0135078430176
Entt Desc. Baro altitude, edited. MSL, low-pass filtered against noise and spikes
Att Unit meters

barom: 2
Max. Value barom :899.485107421875
Min. Value barom :17.0135078430176
Entt Desc. Barometric altitude above MSL. Same as [baltlc]
Att Unit meters

baseB: 3
Max. Value baseB :50856.87890625
Min. Value baseB :50788.08984375
Entt Desc. Main mag base, edited. Edited against cultural signal and noise
Att Unit nanoteslas

baseBo: 4
Max. Value baseBo :51018.91015625
Min. Value baseBo :50339.41796875
Entt Desc. Main mag base, raw	Raw main mag base station
Att Unit nanoteslas

date: 5
Max. Value date :2020/03/07
Min. Value date :2019/10/27
Entt Desc. Acquisition date. Start of acquisition, UTC
Att Unit yyyy/mm/dd

directn: 6
Max. V

In [304]:
#strip Tabs from string
stripFileListing = fileListing.replace('\t', '')
#Create array of values by splitting on line return \n
arrayFileListing = stripFileListing.splitlines()
#arrayFileListing[0] = 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_TMFI.tif'
arrayFileListing

['USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMF.tif',
 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMFI.tif',
 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_TMFI.tif',
 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_VerticalMagneticGradientShaded.tif']

# Now lets create Entity and Attributes for the rest of the files that don't require listing the min/max for the domains.

In [305]:
# Create the Entity and Attributes for the other files being released besides the *.CSV data file
bFilesEntAttText = ''
bFilesEandA = ''
for i in range(len(arrayFileListing)):
    if arrayFileListing[i].find('.tif') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>3-band raster geospatial data file of the aeromagnetic data.</enttypd>' \
            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Value</attrlabl>' \
            + '\n\t\t\t\t<attrdef>Unique numeric values contained in each raster cell.</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<rdom>\n\t\t\t\t\t\t<rdommin>0</rdommin>\n\t\t\t\t\t\t<rdommax>255</rdommax>'\
            + '\n\t\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n' 
    if arrayFileListing[i].find('.rtf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Rich Text File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ReadMe file in Rich Text Format containing *.CSV file entities, attributes, and processing notes</enttypd>' \
            + '\n\t\t\t\t<enttypds>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Rich Text File</attrlabl>\n\t\t\t\t<attrdef>ReadMe file in Rich Text Format (RTF) containing *.CSV file entities, attributes, and processing notes</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>ReadMe file in Rich Text Format containing *.CSV file entities, attributes, and processing notes</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    if arrayFileListing[i].find('.gdb') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Binary File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Binary data base file in Geosoft (https://www.geosoft.com) Oasis montaj GRD format.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Binary data base file</attrlabl>\n\t\t\t\t<attrdef>Binary data base file described in contractor report and ReadMe file</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>Binary Data Base File described in contractor report and ReadMe file</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if arrayFileListing[i].find('.xml') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>ASCII XML File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ASCII text XML formatted metadata file describing the data base file of the same name.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>ASCII XML formatted text file</attrlabl>\n\t\t\t\t<attrdef>ASCII XML formatted text file containing metadata describing the Geosoft data base file of the same name</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>ASCII XML formatted text file containing metadata describing the Geosoft data base file of the same name</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if arrayFileListing[i].find('.gxf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>ASCII Grid File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ASCII grid file in Geosoft (https://www.geosoft.com) Grid Exchange Format, GXF (https://www.geosoft.com/media/uploads/resources/technical-notes/gxfr3d9_1.pdf) </enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>ASCII Grid File</attrlabl>\n\t\t\t\t<attrdef>ASCII grid file in GXF format described in contractor report and ReadMe file, WGS84 UTM Zone 13N</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n'\
            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>ASCII Grid File described in contractor report and ReadMe file, WGS84 UTM Zone 13N</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    
    if arrayFileListing[i].find('.grd') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Binary Grid File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Binary grid file in Geosoft (https://www.geosoft.com) Oasis montaj GRD format.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Binary grid file</attrlabl>\n\t\t\t\t<attrdef>Binary grid file described in contractor report and ReadMe file, WGS84 UTM Zone 13N</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n'\
            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>Binary Grid File described in contractor report and ReadMe file, WGS84 UTM Zone 13N</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    bFilesEntAttText = bFilesEandA

print (bFilesEntAttText) 


		<detailed>
			<enttyp>
				<enttypl>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMF.tif</enttypl>
				<enttypd>3-band raster geospatial data file of the aeromagnetic data.</enttypd>
				<enttypds>U.S. Geological Survey</enttypds>
			</enttyp>
			<attr>
				<attrlabl>Value</attrlabl>
				<attrdef>Unique numeric values contained in each raster cell.</attrdef>
				<attrdefs>U.S. Geological Survey</attrdefs>
				<attrdomv>
					<rdom>
						<rdommin>0</rdommin>
						<rdommax>255</rdommax>
					</rdom>
				</attrdomv>
			</attr>
		</detailed>
		<detailed>
			<enttyp>
				<enttypl>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMFI.tif</enttypl>
				<enttypd>3-band raster geospatial data file of the aeromagnetic data.</enttypd>
				<enttypds>U.S. Geological Survey</enttypds>
			</enttyp>
			<attr>
				<attrlabl>Value</attrlabl>
				<attrdef>Unique numeric values contained in each raster cell.</attrdef>
				<attrdefs>U.S. Geologica

In [306]:
# Run this cell for key word text to edit.  
# One can edit the text in place and these changes will be present in the created metadata.  
# When complete move on to the next step

display(bFilesEntAttText)

'\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMF.tif</enttypl>\n\t\t\t\t<enttypd>3-band raster geospatial data file of the aeromagnetic data.</enttypd>\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Value</attrlabl>\n\t\t\t\t<attrdef>Unique numeric values contained in each raster cell.</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>\n\t\t\t\t\t<rdom>\n\t\t\t\t\t\t<rdommin>0</rdommin>\n\t\t\t\t\t\t<rdommax>255</rdommax>\n\t\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n\t\t\t</attr>\n\t\t</detailed>\n\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_ShadedTMFI.tif</enttypl>\n\t\t\t\t<enttypd>3-band raster geospatial data file of the aeromagnetic data.</enttypd>\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>\n\t\t\t<attr>\n\t\t\t\t<attrlab

In [307]:
#Enter the URL for the magnetic data child for this data release between single quotes below

###############################################################
childURL = 'https://www.sciencebase.gov/catalog/item/5e6c0b09e4b01d5092638c8f'
###############################################################
print (childURL)

https://www.sciencebase.gov/catalog/item/5e6c0b09e4b01d5092638c8f


In [308]:
txtListFormatted = []
txtListFormatted.append('USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_TMFShaded.png')#png file


In [309]:
# Create the Entity and Attributes for the .txt? listing (Note: for each station, replace sciencebase link with correct link)
txtEandA = ''
#As before, we read in the list of files

for i in range(len(txtListFormatted)):
    if txtListFormatted[i].find('png') != -1:

        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>' + txtListFormatted[i] + ' is a Portable Network Graphics (PNG) image associated with the data set.</eaover> '\
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \

    if txtListFormatted[i].find('pdf') != -1:

        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File. ' + txtListFormatted[i] + ' is a header file in ASCII text format for binary Fourier coefficient files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \

    if txtListFormatted[i].find('-SD') != -1:
        
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for synchronous detection cross power files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \

            
    if txtListFormatted[i].find('-TS') != -1:

        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for binary time series files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \

#Add the final .txt listing for the readme file.  
#Be sure to comment this out if there is no readme.txt file included with the data release

##readmeEandA = '\t\t<overview>\n\t\t<eaover>Text File, readme.txt, is a Read Me file describing the naming format of the EDI files and that they may need to be renamed to be imported into certain software packages.</eaover>'\
           ## + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
    
txtEandA = txtEandA #+ readmeEandA
print (txtEandA)
#ALL_EandA = '' #'<eainfo>\n'   
#ALL_EandA = ALL_EandA + txtEandA

		<overview>
			<eaover>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_TMFShaded.png is a Portable Network Graphics (PNG) image associated with the data set.</eaover> 
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e6c0b09e4b01d5092638c8f</eadetcit>
		</overview>



# Now concatenate all the entity and attribute text


In [310]:
eainfo = csvAttributes + bFilesEntAttText + txtEandA
eainfo

'\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_MagneticData.csv</enttypl>\n\t\t\t\t<enttypd>Table containing attribute information associated with the data set.</enttypd>\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>baltl</attrlabl>\n\t\t\t\t\t<attrdef>Baro altitude, raw. MSL, computed from measured pressure and temperature</attrdef>\n\t\t\t\t\t<attrdefs>EON GEOSCIENCES INC.</attrdefs>\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<rdom>\n\t\t\t\t\t\t\t<rdommin>14.9452438354492</rdommin>\n\t\t\t\t\t\t\t<rdommax>899.5800170898441</rdommax>\n\t\t\t\t\t\t\t<attrunit>meters</attrunit>\n\t\t\t\t\t\t</rdom>\n\t\t\t\t\t\t</attrdomv>\n\t\t\t\t</attr>\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>baltlc</attrlabl>\n\t\t\t\t\t<attrdef>Baro altitude, edited. MSL, low-pass filtered against noise and spikes</attrdef>\n\t\t\t\t\t<attrdefs>EON GEOSCIENCES INC.</attrdefs>\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<r

# Populate Metadata Template

In [311]:
#Load XML Metadata Template File and Read It
metaData = os.path.join(magMetaDataTemplatePath, magMetaDataTemplateName)
print ('Metadata path: ' + magMetaDataTemplatePath + '\n')
xmlTemplateFile = open(metaData, 'r')
metaDataContent = xmlTemplateFile.readlines()
print(metaDataContent)
xmlTemplateFile.close()


Metadata path: C:\CurrentWork\DataReleases\NorthwesternArkansas\Metadata

['<?xml version="1.0" encoding="UTF-8"?>\n', '<metadata>\n', '  <idinfo>\n', '    <citation>\n', '      <citeinfo>{origin}\n', '        <pubdate>{pubdate}</pubdate>\n', '        <title>{title}</title>\n', '        <edition>1</edition>\n', '        <geoform>ASCII and Binary Digital Data</geoform>\n', '        <pubinfo>\n', '          <pubplace>Denver, CO</pubplace>\n', '          <publish>U.S. Geological Survey</publish>\n', '        </pubinfo>\n', '        <othercit>{othercit}</othercit>\n', '        <onlink>{onlink}</onlink>\n', '      </citeinfo>\n', '    </citation>\n', '    <descript>\n', '      <abstract>{abstract}</abstract>\n', '      <purpose>{purpose}</purpose>\n', '      <supplinf>{supplinf}</supplinf>\n', '    </descript>\n', '    <timeperd>\n', '      <timeinfo>\n', '        <rngdates>\n', '          <begdate>{begdate}</begdate>\n', '          <enddate>{enddate}</enddate>\n', '        </rngdates>\n', 

### !!! Be sure to enter output file name for the metadat below !!!

In [312]:
# Please enter metadata output file name below between the single quotes
#========================================================================================================================
magMetadataOutputFileName = 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Magnetic.xml'
#========================================================================================================================

In [313]:
# Replace values of current metadata template with the appropriate values.  
# All of this input should have been defined when going through the steps outlined above.
lineString = ''
#newMetaDataContent = metaDataContent
myfilename = magMetaDataTemplatePath + "\\" + magMetadataOutputFileName
xmlFile = open(myfilename,"w+")
print('File exported to:\n'+ myfilename)
#print(keywords.value)
for i in range(len(metaDataContent)):
    lineString = metaDataContent[i]
    if lineString.find('{origin}'):
     lineString = lineString.replace('{origin}',origin)
    
    if lineString.find('{pubdate}'):
     lineString = lineString.replace('{pubdate}', pubdate)    
    
    if lineString.find('{title}'):
     lineString = lineString.replace('{title}', title)
    
    if lineString.find('{othercit}'):
     lineString = lineString.replace('{othercit}', othercit)
    
    if lineString.find('{onlink}'):
     lineString = lineString.replace('{onlink}', onlink)
    
    if lineString.find('{abstract}'):
     lineString = lineString.replace('{abstract}', abstract)
    
    if lineString.find('{purpose}'):
     lineString = lineString.replace('{purpose}', purpose)
    
    if lineString.find('{supplinf}'):
     lineString = lineString.replace('{supplinf}', supplinf)
    
    if lineString.find('{begdate}'):
     lineString = lineString.replace('{begdate}', begdate)
    
    if lineString.find('{enddate}'):
     lineString = lineString.replace('{enddate}', enddate)
    
    if lineString.find('{westbc}'):
     lineString = lineString.replace('{westbc}', str(westbc))

    if lineString.find('{eastbc}'):
     lineString = lineString.replace('{eastbc}', str(eastbc))

    if lineString.find('{northbc}'):
     lineString = lineString.replace('{northbc}', str(northbc))
    
    if lineString.find('{southbc}'):
     lineString = lineString.replace('{southbc}', str(southbc))
   
    if lineString.find('{keywords}'):
     lineString = lineString.replace('{keywords}', str(keywords))
    
    if lineString.find('{accconst}'):
     lineString = lineString.replace('{accconst}', accconst)
    
    if lineString.find('{useconst}'):
     lineString = lineString.replace('{useconst}', useconst)
    
    if lineString.find('{cntperDataContact}'):
     lineString = lineString.replace('{cntperDataContact}', cntperDataContact)
    
    if lineString.find('{cntposDataContact}'):
     lineString = lineString.replace('{cntposDataContact}', cntposDataContact)
    
    if lineString.find('{cntvoiceDataContact}'):
     lineString = lineString.replace('{cntvoiceDataContact}', str(cntvoiceDataContact))
    
    if lineString.find('{cntemailDataContact}'):
     lineString = lineString.replace('{cntemailDataContact}', str(cntemailDataContact))
    
    if lineString.find('{datacred}'):
     lineString = lineString.replace('{datacred}', str(datacred))
    
    if lineString.find('{attraccr}'):
     lineString = lineString.replace('{attraccr}', str(attraccr)) 
    
    if lineString.find('{logic}'):
     lineString = lineString.replace('{logic}', logic)
    
    if lineString.find('{complete}'):
     lineString = lineString.replace('{complete}', str(complete))
    
    if lineString.find('{horizpar}'):
     lineString = lineString.replace('{horizpar}', str(horizpar))
    
    if lineString.find('{vertaccr}'):
     lineString = lineString.replace('{vertaccr}', str(vertaccr))
    
    if lineString.find('{lineage}'):
     lineString = lineString.replace('{lineage}', str(lineage))
    
    if lineString.find('{latres}'):
     lineString = lineString.replace('{latres}', str(latres))
    
    if lineString.find('{longres}'):
     lineString = lineString.replace('{longres}', str(longres))
    
    if lineString.find('{horizdn}'):
     lineString = lineString.replace('{horizdn}', str(horizdn))
    
    if lineString.find('{ellips}'):
     lineString = lineString.replace('{ellips}', str(ellips))
    
    if lineString.find('{semiaxis}'):
     lineString = lineString.replace('{semiaxis}', str(semiaxis))
    
    if lineString.find('{denflat}'):
     lineString = lineString.replace('{denflat}', str(denflat))
   
    if lineString.find('{eainfo}'):
     lineString = lineString.replace('{eainfo}', str(eainfo))
    
    if lineString.find('{distliab}'):
     lineString = lineString.replace('{distliab}', str(distliab))
    
    if lineString.find('{metd}'):
     lineString = lineString.replace('{metd}', str(metd))
    
    if lineString.find('{cntperMetaInfo}'):
     lineString = lineString.replace('{cntperMetaInfo}', str(cntperMetaInfo))
    
    if lineString.find('{cntposMetaInfo}'):
     lineString = lineString.replace('{cntposMetaInfo}', str(cntposMetaInfo))
    
    if lineString.find('{cntvoiceMetaInfo}'):
     lineString = lineString.replace('{cntvoiceMetaInfo}', str(cntvoiceMetaInfo))
    
    if lineString.find('{cntemailMetaInfo}'):
     lineString = lineString.replace('{cntemailMetaInfo}', str(cntemailMetaInfo))    
    
    if lineString.find('{stdorder}'):
     lineString = lineString.replace('{stdorder}', str(stdorder))
    
    else:
     lineString = lineString
    xmlFile.write(lineString)
    
    #print (lineString)
     
    
    
#for r in (metaDataContent):
    #newMetaDataContent = metaDataContent.replace('{title}', drTitle)
    #newMetaDataContent = metaDataContent.replace('{keywords}', keywords.value)
xmlFile.close()

print ('Creation of new metadata file is complete\n\n') 
#Load EDI File and Read It
##checkFile = open(open(myfilename, 'r')
##checkFileContent = checkFile.read()
##checkFile.close()
##print(checkFileContent)

File exported to:
C:\CurrentWork\DataReleases\NorthwesternArkansas\Metadata\USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Magnetic.xml
Creation of new metadata file is complete




### Check this file to see if it is valid against the FGDC metadata standard (FGDC-STD-001-1998)

## https://mrdata.usgs.gov/validation/